In [1]:
"""
jupyter-nbconvert tabulate.ipynb --to python 
ipython tabulate.py -- --base_dir ~/plume/plumezoo/latest/fly/memory/

ipython tabulate.py -- --base_dir ~/plume/plumezoo/
ipython tabulate.py -- --base_dir ~/plume/plume2/ppo/trained_models/
"""

'\njupyter-nbconvert tabulate.ipynb --to python \nipython tabulate.py -- --base_dir ~/plume/plumezoo/latest/fly/memory/\n\nipython tabulate.py -- --base_dir ~/plume/plumezoo/\nipython tabulate.py -- --base_dir ~/plume/plume2/ppo/trained_models/\n'

In [2]:
import pandas as pd
import glob 
from pathlib import Path
import tqdm
import os
import argparse

In [3]:
import sys
batchmode = False
if 'ipykernel_launcher' in sys.argv[0]:
    print("Interactive mode")
    
#     BASE_DIR="~/plume/plumezoo/latest/fly/memory/"
    BASE_DIR="~/plume/plumezoo/latest/fly/memory_reject/"
#     BASE_DIR="~/plume/plumezoo/latest/fly/memory_extras/"
#     BASE_DIR="~/plume/plume2/ppo/trained_models/"
#     BASE_DIR="~/plume/plumezoo/"
#     BASE_DIR="~/plume/plumezoo/dynamic/fly/memory_reject/"

    
else:
    batchmode = True
    print("Batch/CLI mode")
    import argparse
    
    parser = argparse.ArgumentParser(description='')
    parser.add_argument('--base_dir', default='./')
    args = parser.parse_args()
    BASE_DIR=args.base_dir

Interactive mode


In [4]:
print(BASE_DIR)
# fnames = Path(BASE_DIR).rglob('*VRNN*/*_summary.csv')
# fnames = !find {BASE_DIR} -name "*VRNN*/*_summary.csv"
fnames = !find {BASE_DIR} -name "*_summary.csv"
# fnames

~/plume/plumezoo/dynamic/fly/memory_reject/


In [5]:
counts_df = []
for fname in tqdm.tqdm(fnames):
    s = pd.read_csv(fname) 
    dataset = str(fname).split('/')[-1].replace('_summary.csv','')
    row = {
        'dataset': dataset,
        'HOME': sum(s['reason'] == 'HOME'),
        'OOB': sum(s['reason'] == 'OOB'),
        'OOT': sum(s['reason'] == 'OOT'),
        'total': len(s['reason']),
        'seed': str(fname).split('seed')[-1].split('/')[0],
        'model_dir': str(fname).replace(f'{dataset}_summary.csv','').replace(os.path.expanduser(BASE_DIR), ''),
        'code': str(fname).split('code')[-1].split('_')[0],
        'fname': str(fname)
    }
    counts_df.append(row)
    
counts_df = pd.DataFrame(counts_df)

100%|██████████| 2759/2759 [00:04<00:00, 638.88it/s]


In [6]:
eligible_datasets = [
        'constantx5b5', 
        'switch15x5b5', 
        'switch30x5b5', 
        'switch45x5b5', 
        'noisy3x5b5', 
        'noisy6x5b5',
        'constantx5b5_0.8',
        'constantx5b5_0.6',
        'constantx5b5_0.4', 
        'constantx5b5_0.2',
]

In [7]:
print(counts_df.shape)
# counts_df = counts_df.query("total == 240 and dataset in @eligible_datasets")
counts_df = counts_df.query("dataset in @eligible_datasets")
print(counts_df.shape)

(2759, 9)
(2651, 9)


In [8]:
counts_df['dataset'].unique()

array(['noisy6x5b5', 'switch45x5b5', 'constantx5b5_0.8',
       'constantx5b5_0.4', 'switch30x5b5', 'constantx5b5_0.6',
       'noisy3x5b5', 'switch15x5b5', 'constantx5b5', 'constantx5b5_0.2'],
      dtype=object)

In [9]:
# Pivot and total
pivot_df = counts_df.pivot(index=['model_dir', 'code'], columns='dataset', values='HOME').reset_index()
# pivot_df['total'] = pivot_df.sum(axis=1, skipna=True)
pivot_df['total'] = pivot_df[['constantx5b5', 'noisy3x5b5', 'switch45x5b5',]].sum(axis=1, skipna=True)
# pivot_df['total'] = pivot_df['constantx5b5']/3 + pivot_df['noisy3x5b5'] + pivot_df['switch45x5b5']

In [10]:
col_order = [
        'total',
        'constantx5b5', 
        'switch45x5b5', 
        'noisy3x5b5', 
        'noisy6x5b5',
        'switch30x5b5', 
        'switch15x5b5', 
        'constantx5b5_0.8',
        'constantx5b5_0.6',
        'constantx5b5_0.4', 
        'constantx5b5_0.2',
        'code',
        'model_dir',
]
pivot_df = pivot_df[col_order]
pivot_df = pivot_df.sort_values(by='total', ascending=False)
pivot_df

dataset,total,constantx5b5,switch45x5b5,noisy3x5b5,noisy6x5b5,switch30x5b5,switch15x5b5,constantx5b5_0.8,constantx5b5_0.6,constantx5b5_0.4,constantx5b5_0.2,code,model_dir
335,274.0,118.0,107.0,49.0,1.0,62.0,34.0,103.0,103.0,103.0,103.0,MLPs10,plume_20210529_MLP_s10_noisy3x5b5_stepoob_fixe...
65,268.0,128.0,17.0,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRNN,plume_20210523_VRNN_constantx5b5_step_posend_b...
119,253.0,82.0,118.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRNN,plume_20210524_VRNN_constantx5b5_step_posend_b...
28,244.0,155.0,73.0,16.0,31.0,104.0,127.0,130.0,137.0,6.0,0.0,VRNN,plume_20210521_VRNN_constantx5b5_stepstray_del...
127,233.0,153.0,77.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRNN,plume_20210524_VRNN_constantx5b5_stepend_bx0.3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MLPs10,plume_20210529_MLP_s10_noisy3x5b5_stepoob_fixe...
333,0.0,0.0,0.0,0.0,50.0,0.0,0.0,5.0,5.0,5.0,5.0,MLPs10,plume_20210529_MLP_s10_noisy3x5b5_stepoob_fixe...
303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MLPs02,plume_20210529_MLP_s02_noisy3x5b5_stepoob_fixe...
178,0.0,0.0,0.0,0.0,2.0,NaN,NaN,2.0,2.0,2.0,2.0,MLPs04,plume_20210526_MLP_s04_constantx5b5_step_posoo...


In [11]:
pivot_df.to_csv(f'{BASE_DIR}/tabulated.tsv', sep='\t', index=False)
pivot_df.to_csv(f'{BASE_DIR}/tabulated.csv', index=False)

In [12]:
# import seaborn as sns
# sns.pairplot(pivot_df)

In [13]:
# Before filtering
pivot_df.groupby('code')['total'].count()

code
GRU        8
MLPs02    46
MLPs04    39
MLPs06    39
MLPs08    45
MLPs10    44
MLPs12    52
VRNN      89
Name: total, dtype: int64

In [21]:
# Filter for quality
# pivot_df.query("constantx5b5>0 and switch45x5b5>0 and noisy3x5b5>0 ").groupby('code').head(5)
# nonzeros = pivot_df.query("constantx5b5>0 and switch45x5b5>0 and noisy3x5b5>0 ")
# nonzeros = pivot_df.query("constantx5b5>0 and switch45x5b5>0 ") # for MLP_s02 and MLP_s04
nonzeros = pivot_df.query("constantx5b5>0") # for MLP_s02 and MLP_s04

In [22]:
# Sort to select top
selected = nonzeros.sort_values(by='total', ascending=False).groupby('code').head(5) # .reset_index(drop=True)
selected.groupby('code')['total'].count()

code
GRU       5
MLPs02    5
MLPs04    5
MLPs06    5
MLPs08    5
MLPs10    5
MLPs12    5
VRNN      5
Name: total, dtype: int64

In [23]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
selected['model_dir'].to_csv(f'{BASE_DIR}/selected.csv', index=False, header=False)
selected['model_dir']

335    plume_20210529_MLP_s10_noisy3x5b5_stepoob_fixe...
65     plume_20210523_VRNN_constantx5b5_step_posend_b...
119    plume_20210524_VRNN_constantx5b5_step_posend_b...
28     plume_20210521_VRNN_constantx5b5_stepstray_del...
127    plume_20210524_VRNN_constantx5b5_stepend_bx0.3...
242    plume_20210527_VRNN_constantx5b5_stepoob_fixed...
105    plume_20210524_MLP_s10_constantx5b5_stepend_bx...
236    plume_20210527_MLP_s12_constantx5b5_step_posoo...
113    plume_20210524_MLP_s12_constantx5b5_stepend_bx...
213    plume_20210527_GRU_constantx5b5_stepoob_fixed_...
211    plume_20210527_GRU_constantx5b5_stepoob_fixed_...
191    plume_20210526_MLP_s10_constantx5b5_step_posoo...
187    plume_20210526_MLP_s08_constantx5b5_step_posoo...
70     plume_20210524_MLP_s02_constantx5b5_step_posen...
78     plume_20210524_MLP_s04_constantx5b5_step_posen...
57     plume_20210523_MLP_s10_constantx5b5_stepend_bx...
208    plume_20210527_GRU_constantx5b5_step_posoob_fi...
8      plume_20210520_MLP_s12_n

In [17]:
"""
mkdir ../memory/

DESTDIR=../memory/
for ARCH in _s02 _s04 _s06 _s08 _s10 _s12 VRNN GRU; do
	for DIR in $(cat selected.csv | grep $ARCH); do 
	 echo ${DIR%/} $(ls ${DIR%/}* | wc -l)
	 mv ${DIR%/}* $DESTDIR
	done
done

for ARCH in _s02 _s04; do
	for DIR in $(cat selected.csv | grep $ARCH); do 
	 echo ${DIR%/} $(ls ${DIR%/}* | wc -l)
	 mv ${DIR%/}* $DESTDIR
	done
done
"""

'\nDESTDIR=../memory/\nfor ARCH in _s06 _s08 _s10 _s12 VRNN GRU; do\n\tfor DIR in $(cat selected.csv | grep $ARCH); do \n\t echo ${DIR%/} $(ls ${DIR%/}* | wc -l)\n\t mv ${DIR%/}* $DESTDIR\n\tdone\ndone\n\nfor ARCH in _s02 _s04; do\n\tfor DIR in $(cat selected.csv | grep $ARCH); do \n\t echo ${DIR%/} $(ls ${DIR%/}* | wc -l)\n\t mv ${DIR%/}* $DESTDIR\n\tdone\ndone\n'